In [ ]:
!pip -q install langchain huggingface_hub   lancedb openai  tiktoken  rank_bm25 pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# pass openai api key
import os
os.environ["OPENAI_API_KEY"] = "sk-your api key"

#if you want to use opensource models such as lama,mistral check this
https://github.com/lancedb/vectordb-recipes/blob/main/tutorials/chatbot_using_Llama2_&_lanceDB

also you can compaire your results with normal alone retriver vs ensamble retriver


# Hybrid Search

## BM25 Retriever - Sparse retriever
## Embeddings - Dense retrievers Lancedb

hubrid serch = sparse + Dense retriver combinig

In [ ]:
# download the pdf
!wget https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf

--2023-12-08 10:04:09--  https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf
Resolving pdf.usaid.gov (pdf.usaid.gov)... 184.29.214.144, 2600:1406:5e00:180::1923, 2600:1406:5e00:1a4::1923
Connecting to pdf.usaid.gov (pdf.usaid.gov)|184.29.214.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6419525 (6.1M) [application/pdf]
Saving to: ‘PA00TBCT.pdf.1’

PA00TBCT.pdf.1      100%[===================>]   6.12M  16.9MB/s    in 0.4s    

2023-12-08 10:04:10 (16.9 MB/s) - ‘PA00TBCT.pdf.1’ saved [6419525/6419525]



In [ ]:

# example of using bm25 & lancedb -hybrid serch


from langchain.vectorstores import LanceDB
import lancedb
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# Initialize embeddings
embedding = OpenAIEmbeddings()


# load single pdf

loader = PyPDFLoader("/content/PA00TBCT.pdf")
pages = loader.load_and_split()



# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(pages)
bm25_retriever.k =  2  # Retrieve top 2 results


print("type of bm25",type(bm25_retriever))



db = lancedb.connect('/tmp/lancedb')
table = db.create_table("pandas_docs", data=[
    {"vector": embedding.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")
# docsearch = LanceDB.from_texts(doc_list, embedding, connection=table)
# retriever_lancedb = docsearch.as_retriever(search_kwargs={"k": 2})


# Initialize LanceDB retriever
docsearch = LanceDB.from_documents(pages, embedding, connection=table)
retriever_lancedb = docsearch.as_retriever(search_kwargs={"k": 2})

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_lancedb],
                                       weights=[0.2, 0.8])

# Example customer query
query = "what nutrition needed for pregnant women ?"


# Retrieve relevant documents/products
docs = ensemble_retriever.get_relevant_documents(query)

# Extract and print only the page content from each document
# for doc in docs:
#     print(doc.page_content)

docs

type of bm25 <class 'langchain.retrievers.bm25.BM25Retriever'>


[Document(page_content='Food and Nutrition Handbook for Extension Workers34guidelines for selecting energy-giving foods, body-building foods \nand protective foods. Pregnant women especially need foods rich in \niron and vitamin A in addition to the balanced diet. Iron needs are \nhighly increased partly due to the need to build reserves for child \nup to six months after birth before initiating complementary food \nintake.\n• Pregnant women need to take foods rich in calcium, e.g., milk and mukene (silver fish) partly to take care of the increased requirement \nfor building the foetus skeletal structure.\n• Pregnant women have higher needs for nutrients generally and \nshould take snacks in between meals.\nIn addition, pregnant women should be educated to strictly observe the \nfollowing:\n1. Take the required amounts of iron and folic acid supplements to \nprevent anaemia.\n2. Sleep under an insecticide-treated mosquito net.\n3. Visit the nearest health facility at least four (4) tim

In [ ]:
#you can ask quetions on this retriver doc
from langchain.chains import RetrievalQA

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key="sk-")

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=ensemble_retriever)


query = "what nutrition needed for pregnant women  "
qa.run(query)


"Pregnant women need to consume a balanced diet that includes a variety of foods from the main food groups. They should also have one additional meal in addition to the three meals they receive daily. Some specific nutrients that are important for pregnant women include:\n\n1. Iron: Pregnant women should consume foods rich in iron, such as lean meats, poultry, fish, beans, and leafy green vegetables. They may also need to take iron supplements as prescribed by a healthcare professional.\n\n2. Folate: Pregnant women should take folate tablets daily in addition to consuming foods rich in folate, such as dark leafy greens, citrus fruits, beans, and fortified grains.\n\n3. Calcium: Foods rich in calcium, such as milk, yogurt, cheese, and fortified plant-based milk alternatives, are important for the development of the baby's skeletal structure.\n\n4. Vitamin A: Pregnant women should consume foods rich in vitamin A, such as carrots, sweet potatoes, spinach, and mangoes. They may also need t

In [ ]:
query = "which food needed for building strong bones and teeth ? which Vitamin & minerals importat for this? "
qa.run(query)


"The foods that are important for building strong bones and teeth are:\n\n1. Calcium: Milk and milk products, fish with bones (such as sardines), dark green vegetables (such as broccoli and kale).\n\n2. Vitamin D: Sun exposure is the primary source, but it can also be found in vitamin D-fortified milk, eggs, fish-liver oils, and fatty fish like herring, mackerel, and salmon.\n\n3. Vitamin K: Vegetables such as spinach, lettuce, cauliflower, and cabbage, broccoli, fish, liver, meat, and eggs.\n\n4. Magnesium: Legumes, whole-grain cereals, nuts, and dark-green vegetables, as well as meat, seafood, and dairy products.\n\n5. Fluoride: Seafood, tea, coffee, soybeans, and sodium fluoride (which is often added to the water supply).\n\n6. Iron: Red meat, organ meats, whole-wheat products, shellfish, nuts, and dried fruit.\n\nIt's important to note that these nutrients work together and are best obtained through a balanced diet."

## additional  bonus
## FTS is another importat feature for extracting all info if any one word is matching

usecase : E-Commerce Product Search
Context: Customers searching for products on an e-commerce website.
Application: When a customer types a query (like "fitness t-shirt"), the system uses the ensemble retriever to find the most relevant products from the product descriptions. The BM25 component helps capture keyword-based matches, while the dense vector retriever (LanceDB) understands the semantic context of the query.

In [ ]:
!pip install tantivy==0.20.1


read more about fts https://lancedb.github.io/lancedb/fts/

In [ ]:
#example of FTS. when you want to serch whole text



import lancedb

uri = "data/sample-lancedb"
db = lancedb.connect(uri)

table = db.create_table("my_tableasd",
            data=[{"vector": [3.1, 4.1], "text": "Frodo was a happy puppy"},
                  {"vector": [5.9, 26.5], "text": "There are several kittens playing"}])


table.create_fts_index("text")


x= table.search("puppy").limit(10).select(["text"]).to_list()


texts = [item['text'] for item in x]
print(texts)
